In [30]:
import yaml
import pickle
import random
import numpy as np
from matplotlib import pylab as plt
import plotly
import plotly.offline as py
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
plotly.offline.init_notebook_mode(connected=True)

#from model.taskHelper import taskHelper
from preprocessing.dataset import mDataset
from preprocessing.featureSelector import featureSelector

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, AdaBoostClassifier, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.neural_network import MLPClassifier

from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [31]:
# Hyper-parameters
cfg = yaml.load(open("./cfg.yaml", "r"), Loader=yaml.FullLoader)
class_names = cfg["class_names"]
feature_names = cfg["feature_names"]
feature_dir = cfg["feature_dir"]

In [32]:
# Configure
show_feature_name = False
feature_dir1 = "/data1/lzy/3st/features_delta_exp_1mm/"
feature_dir2 = "/data1/lzy/3st/features_delta_shr_1mm/"
class_names_train = {
    "C058_train_origin_delta": 0,
    "C058_train_exp1_delta": 1,
    "C058_train_shr1_delta": -1,
    "C058_train_exp2_delta": 2,
    "C058_train_shr2_delta": -2,
    "C058_train_exp3_delta": 3,
    "C058_train_shr3_delta": -3,
    "C058_train_exp4_delta": 4,
    "C058_train_shr4_delta": -4,
}
class_names_test = {
    "C058_test_origin_delta": 0,
    "C058_test_exp1_delta": 1,
    "C058_test_shr1_delta": -1,
    "C058_test_exp2_delta": 2,
    "C058_test_shr2_delta": -2,
    "C058_test_exp3_delta": 3,
    "C058_test_shr3_delta": -3,
    "C058_test_exp4_delta": 4,
    "C058_test_shr4_delta": -4,
}

In [33]:
myDataset = mDataset(feature_dir1, class_names_train, feature_names)
X_train1, y_train1 = myDataset.getDataset()
myDataset = mDataset(feature_dir2, class_names_train, feature_names)
X_train2, y_train2 = myDataset.getDataset()

myDataset = mDataset(feature_dir1, class_names_test, feature_names)
X_test1, y_test1 = myDataset.getDataset()
myDataset = mDataset(feature_dir2, class_names_test, feature_names)
X_test2, y_test2 = myDataset.getDataset()

X_train = np.concatenate([X_train1,X_train2],axis=1)
y_train = y_train1
X_test = np.concatenate([X_test1,X_test2],axis=1)
y_test = y_test1

print(X_train.shape)
print(X_test.shape)

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(X_train)
X_train = imp.transform(X_train)
X_test = imp.transform(X_test)


scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

nor = Normalizer()
nor.fit(X_train)
X_train = nor.transform(X_train)
X_test = nor.transform(X_test)

mms = MinMaxScaler()
mms.fit(X_train)
X_train = mms.transform(X_train)
X_test = mms.transform(X_test)


get # 112  of  C058_train_origin_delta
get # 112  of  C058_train_exp1_delta
get # 112  of  C058_train_shr1_delta
get # 112  of  C058_train_exp2_delta
get # 112  of  C058_train_shr2_delta
get # 112  of  C058_train_exp3_delta
get # 112  of  C058_train_shr3_delta
get # 112  of  C058_train_exp4_delta
get # 112  of  C058_train_shr4_delta
get # 112  of  C058_train_origin_delta
get # 112  of  C058_train_exp1_delta
get # 112  of  C058_train_shr1_delta
get # 112  of  C058_train_exp2_delta
get # 112  of  C058_train_shr2_delta
get # 112  of  C058_train_exp3_delta
get # 112  of  C058_train_shr3_delta
get # 112  of  C058_train_exp4_delta
get # 112  of  C058_train_shr4_delta
get # 87  of  C058_test_origin_delta
get # 87  of  C058_test_exp1_delta
get # 87  of  C058_test_shr1_delta
get # 87  of  C058_test_exp2_delta
get # 87  of  C058_test_shr2_delta
get # 87  of  C058_test_exp3_delta
get # 87  of  C058_test_shr3_delta
get # 87  of  C058_test_exp4_delta
get # 87  of  C058_test_shr4_delta
get # 87  of 

In [34]:
def train_test(X_new, y_new, X_test, y_test):
#     model = RandomForestClassifier(n_estimators=10, max_depth=7, max_features='sqrt', random_state=0)
#     model.fit(X_new, y_new)
#     print("RF mean accuracy on testset:", model.score(X_test, y_test))
#     model2 = AdaBoostClassifier(n_estimators=100, random_state=0)
#     model2.fit(X_new, y_new)
#     print("AdaBoost mean accuracy on testset:", model2.score(X_test, y_test))
    model2 = BaggingRegressor(n_estimators=20)
    model2.fit(X_new, y_new)
    guesses = model2.predict(X_test)
    error = mean_squared_error(y_test, guesses)
    error_mae = mean_absolute_error(y_test, guesses)
    print("BaggingRegressor MSE :", error)
    print("BaggingRegressor MAE :", error_mae)

In [35]:
feature_name_all = []
feature_names_new = np.asarray(feature_names + feature_names)

In [36]:
X_new, y_new = X_train, y_train
X_test111, y_test111 = X_test, y_test
slor = featureSelector(X_new, y_new)
X_new, y_new = slor.corrSelector(method_name="pearson", num=1000)
print("train set shape: ", X_new.shape)

indexs = slor.indexs
X_test111 = X_test111[:, indexs]
print("test set shape: ", X_test111.shape)

# feature_names_new = feature_names_new[slor.indexs]
# feature_name_all.append(feature_names_new)
# print(feature_names_new)

train_test(X_new, y_new, X_test111, y_test111)


Feature selecting method:  pearson
features dim:  1800
train set shape:  (1008, 1000)
test set shape:  (783, 1000)
BaggingRegressor MSE : 1.3639208173690933
BaggingRegressor MAE : 0.6939974457215836


In [24]:
# for k in range(500,1800,100):
#     print(k)
#     X_new, y_new = X_train, y_train
#     X_test111, y_test111 = X_test, y_test
#     slor = featureSelector(X_new, y_new)
#     X_new, y_new = slor.univarSelector(top_k=k)
#     print("train set shape: ", X_new.shape)

#     indexs = slor.indexs
#     X_test111 = X_test111[:, indexs]
#     print("test set shape: ", X_test111.shape)

#     # feature_names_new = feature_names_new[slor.indexs]
#     # feature_name_all.append(feature_names_new)
#     # print(feature_names_new)

#     train_test(X_new, y_new, X_test111, y_test111)


In [25]:
k=600
slor = featureSelector(X_new, y_new)
X_new, y_new = slor.univarSelector(top_k=k)
print("train set shape: ", X_new.shape)

indexs = slor.indexs
X_test111 = X_test111[:, indexs]
print("test set shape: ", X_test111.shape)

feature_names_new = feature_names_new[slor.indexs]
feature_name_all.append(feature_names_new)
print(feature_names_new)

train_test(X_new, y_new, X_test111, y_test111)


Feature selecting method:  f_classif
train set shape:  (1008, 600)
test set shape:  (783, 600)
['original_glcm_Autocorrelation' 'original_glcm_ClusterProminence'
 'original_glcm_ClusterShade' 'original_glcm_ClusterTendency'
 'original_glcm_Contrast' 'original_glcm_Correlation'
 'original_glcm_DifferenceAverage' 'original_glcm_DifferenceEntropy'
 'original_glcm_DifferenceVariance' 'original_glcm_Id' 'original_glcm_Idm'
 'original_glcm_Idmn' 'original_glcm_Idn' 'original_glcm_Imc1'
 'original_glcm_JointAverage' 'original_glcm_JointEntropy'
 'original_glcm_MCC' 'original_glcm_SumAverage' 'original_glcm_SumEntropy'
 'original_glcm_SumSquares' 'original_glrlm_GrayLevelVariance'
 'original_glrlm_LongRunEmphasis' 'original_glrlm_RunEntropy'
 'original_glrlm_RunLengthNonUniformity'
 'original_glrlm_RunLengthNonUniformityNormalized'
 'original_glrlm_RunPercentage' 'original_glrlm_ShortRunEmphasis'
 'original_glrlm_ShortRunHighGrayLevelEmphasis'
 'original_glszm_GrayLevelNonUniformity'
 'origina

BaggingRegressor MSE : 1.4066602809706261
BaggingRegressor MAE : 0.5947637292464879


In [37]:
slor = featureSelector(X_new, y_new)
X_new, y_new = slor.ttestSelector()
print("train set shape: ", X_new.shape)

indexs = slor.indexs
X_test111 = X_test111[:, indexs]
print("test set shape: ", X_test111.shape)

feature_names_new = feature_names_new[slor.indexs]
feature_name_all.append(feature_names_new)
print(feature_names_new)

train_test(X_new, y_new, X_test111, y_test111)


Feature selecting method: T test
train set shape:  (1008, 252)
test set shape:  (783, 252)
['original_glrlm_HighGrayLevelRunEmphasis'
 'original_glrlm_RunLengthNonUniformity'
 'original_glrlm_RunLengthNonUniformityNormalized'
 'original_glrlm_ShortRunHighGrayLevelEmphasis'
 'original_gldm_LargeDependenceHighGrayLevelEmphasis'
 'log-sigma-1-0-mm-3D_glcm_ClusterProminence'
 'log-sigma-1-0-mm-3D_glcm_ClusterShade'
 'log-sigma-1-0-mm-3D_glcm_MaximumProbability'
 'log-sigma-1-0-mm-3D_glrlm_ShortRunLowGrayLevelEmphasis'
 'log-sigma-1-0-mm-3D_glszm_GrayLevelNonUniformity'
 'log-sigma-1-0-mm-3D_glszm_GrayLevelNonUniformityNormalized'
 'log-sigma-1-0-mm-3D_glszm_LowGrayLevelZoneEmphasis'
 'log-sigma-1-0-mm-3D_glszm_SmallAreaLowGrayLevelEmphasis'
 'log-sigma-1-0-mm-3D_gldm_DependenceVariance'
 'log-sigma-1-0-mm-3D_gldm_GrayLevelNonUniformity'
 'log-sigma-1-0-mm-3D_gldm_GrayLevelVariance'
 'log-sigma-1-0-mm-3D_ngtdm_Complexity'
 'log-sigma-1-0-mm-3D_ngtdm_Strength'
 'log-sigma-3-0-mm-3D_glcm_Diff

BaggingRegressor MSE : 1.3401979565772668
BaggingRegressor MAE : 0.6601532567049808


In [38]:
slor = featureSelector(X_new, y_new)
X_new, y_new = slor.mannSelector()
print("train set shape: ", X_new.shape)

indexs = slor.indexs
X_test111 = X_test111[:, indexs]
print("test set shape: ", X_test111.shape)

feature_names_new = feature_names_new[slor.indexs]
feature_name_all.append(feature_names_new)
print(feature_names_new)

train_test(X_new, y_new, X_test111, y_test111)


Feature selecting method: mann whitney u test
train set shape:  (1008, 232)
test set shape:  (783, 232)
['original_glrlm_HighGrayLevelRunEmphasis'
 'original_glrlm_RunLengthNonUniformityNormalized'
 'original_glrlm_ShortRunHighGrayLevelEmphasis'
 'original_gldm_LargeDependenceHighGrayLevelEmphasis'
 'log-sigma-1-0-mm-3D_glcm_ClusterProminence'
 'log-sigma-1-0-mm-3D_glcm_ClusterShade'
 'log-sigma-1-0-mm-3D_glcm_MaximumProbability'
 'log-sigma-1-0-mm-3D_glrlm_ShortRunLowGrayLevelEmphasis'
 'log-sigma-1-0-mm-3D_glszm_GrayLevelNonUniformity'
 'log-sigma-1-0-mm-3D_glszm_GrayLevelNonUniformityNormalized'
 'log-sigma-1-0-mm-3D_glszm_LowGrayLevelZoneEmphasis'
 'log-sigma-1-0-mm-3D_glszm_SmallAreaLowGrayLevelEmphasis'
 'log-sigma-1-0-mm-3D_gldm_DependenceVariance'
 'log-sigma-1-0-mm-3D_gldm_GrayLevelNonUniformity'
 'log-sigma-1-0-mm-3D_gldm_GrayLevelVariance'
 'log-sigma-1-0-mm-3D_ngtdm_Strength'
 'log-sigma-3-0-mm-3D_glcm_DifferenceAverage'
 'log-sigma-3-0-mm-3D_glcm_DifferenceVariance'
 'log-

BaggingRegressor MSE : 1.2319029374201789
BaggingRegressor MAE : 0.6545338441890166


In [39]:
slor = featureSelector(X_new, y_new)
X_new, y_new = slor.modelSelector(model_name="rf")
print("train set shape: ", X_new.shape)

indexs = slor.indexs
X_test111 = X_test111[:, indexs]
print("test set shape: ", X_test111.shape)

feature_names_new = feature_names_new[slor.indexs]
feature_name_all.append(feature_names_new)
print(feature_names_new)

train_test(X_new, y_new, X_test111, y_test111)


Feature selecting method:  rf
train set shape:  (1008, 52)
test set shape:  (783, 52)
['log-sigma-1-0-mm-3D_glcm_MaximumProbability'
 'log-sigma-1-0-mm-3D_glrlm_ShortRunLowGrayLevelEmphasis'
 'log-sigma-1-0-mm-3D_gldm_GrayLevelNonUniformity'
 'log-sigma-1-0-mm-3D_gldm_GrayLevelVariance' 'wavelet-LLH_glcm_Idn'
 'wavelet-LHL_glcm_Contrast' 'wavelet-LHL_glcm_SumEntropy'
 'wavelet-LHL_glszm_LargeAreaLowGrayLevelEmphasis'
 'wavelet-LHL_glszm_SmallAreaLowGrayLevelEmphasis'
 'wavelet-LHH_glcm_Autocorrelation' 'wavelet-LHH_glcm_ClusterProminence'
 'wavelet-LHH_glrlm_HighGrayLevelRunEmphasis'
 'wavelet-LHH_glrlm_LongRunEmphasis' 'wavelet-HLL_glcm_Idm'
 'wavelet-HLL_glcm_JointEnergy' 'wavelet-HLL_glszm_LargeAreaEmphasis'
 'wavelet-HLL_gldm_DependenceNonUniformity'
 'wavelet-HLL_gldm_GrayLevelVariance' 'wavelet-HLH_glcm_Contrast'
 'wavelet-HLH_glcm_Correlation' 'wavelet-HLH_glrlm_LongRunEmphasis'
 'wavelet-HLH_glrlm_LowGrayLevelRunEmphasis'
 'wavelet-HLH_gldm_LowGrayLevelEmphasis' 'wavelet-HLH_ng

In [40]:
slor = featureSelector(X_new, y_new)
X_new, y_new = slor.modelSelector(model_name="lasso")
print("train set shape: ", X_new.shape)

indexs = slor.indexs
X_test111 = X_test111[:, indexs]
print("test set shape: ", X_test111.shape)

feature_names_new = feature_names_new[slor.indexs]
feature_name_all.append(feature_names_new)
print(feature_names_new)

train_test(X_new, y_new, X_test111, y_test111)


Feature selecting method:  lasso


/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.1482487407988629, tolerance: 0.5681995037220844

/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.198277208122363, tolerance: 0.5681995037220844

/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.1675555501576582, tolerance: 0.5681995037220844

/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 

train set shape:  (1008, 15)
test set shape:  (783, 15)
['log-sigma-1-0-mm-3D_glcm_MaximumProbability'
 'log-sigma-1-0-mm-3D_gldm_GrayLevelNonUniformity'
 'wavelet-LHL_glcm_Contrast' 'wavelet-LHH_glcm_Autocorrelation'
 'wavelet-LHH_glrlm_LongRunEmphasis' 'wavelet-HLL_glszm_LargeAreaEmphasis'
 'wavelet-HLL_gldm_GrayLevelVariance'
 'wavelet-HLH_glrlm_LowGrayLevelRunEmphasis'
 'wavelet-HHL_glrlm_LowGrayLevelRunEmphasis'
 'wavelet-HHL_glrlm_RunLengthNonUniformity' 'wavelet-HHH_glcm_Idmn'
 'wavelet-HHH_glcm_Imc1' 'wavelet-LLL_glcm_JointEntropy'
 'wavelet-LLL_glcm_SumEntropy' 'wavelet-LLL_ngtdm_Coarseness']
BaggingRegressor MSE : 1.9098371647509575
BaggingRegressor MAE : 0.7738825031928481
